In [5]:
import os 
from dotenv import load_dotenv
load_dotenv()

False

In [ ]:
## Setting up llama2 as model
MODEL = 'llama2'


In [13]:
## Make sure you have Ollama and llama2 installed locally
from langchain_community.llms import Ollama
model = Ollama(model=MODEL)

## Invoke and test if its working
model.invoke("hey tell me a joke")


"\nSure, here's one:\n\nWhy don't scientists trust atoms?\nBecause they make up everything!\n\nI hope that made you smile! Do you want to hear another one?"

In [ ]:
## Loading my pdf into loader and splitting it into pages
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("mycv.pdf")
pages = loader.load_and_split()
pages


[Document(metadata={'source': 'mycv.pdf', 'page': 0}, page_content='Shrikrushna Gundre\n/gl⌢beshricastic.in ♂¶obile7378484905 /envel⌢peshricse07@gmail.com /linkedinshricastic /githubshricastic /codeleetcode\nEducation\nKolhapur Institutet of Technology 2021 - 2024\nB.tech in Computer Science and Engineering (GPA:7.9) Kolhapur, Maharashtra\nTechnical Skills\nLanguages: C++, TypeScript, Python, JavaScript, Rust, Java, Solidity, Html, Css\nTechnologies/Frameworks: NextJs, React, Express, NodeJs, FastApi, Recoil, Tailwind CSS, Prisma(ORM)\nDatabases: MongoDB, PostgreSQL, MySQL, Redis\nOther Skills: AWS, GCP, Docker, Git, Linux, Bash Shell, Blockchain, Postman, Serverless\nExperience\nTecell Oct 2024 – Present\nSoftware Engineer - Intern Bengaluru, Karnataka\n• Focusing on Node and cloud application integration to develop software that connects and manages EV charging\nstations.\n• Contributing to projects aimed at enhancing system performance and scalability within the EV infrastructure.\n

In [ ]:
## Create prompt template for the llm
from langchain.prompts import PromptTemplate

template= """ 
Answer the questions based on the context below. If you cant 
answer the question, reply "I dont know"

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Some context", question="here is the question"))

 
Answer the questions based on the context below. If you cant 
answer the question, reply "I dont know"

Context: Some context

Question: here is the question



In [ ]:
#create chain and test by invoking it
chain = prompt | model

chain.invoke(
    {
        "context": "I am shrikrushna",
        "question": "whats my name?"
    }
)

' Sure! As Shrikrushna, my answer to your question would be: "Your name is... Krishna!"'

In [40]:
## Create embeddings using langchain and Ollama
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [41]:
retriever = vectorstore.as_retriever()

retriever.invoke("Shrikrushna")

[Document(metadata={'source': 'mycv.pdf', 'page': 0}, page_content='Shrikrushna Gundre\n/gl⌢beshricastic.in ♂¶obile7378484905 /envel⌢peshricse07@gmail.com /linkedinshricastic /githubshricastic /codeleetcode\nEducation\nKolhapur Institutet of Technology 2021 - 2024\nB.tech in Computer Science and Engineering (GPA:7.9) Kolhapur, Maharashtra\nTechnical Skills\nLanguages: C++, TypeScript, Python, JavaScript, Rust, Java, Solidity, Html, Css\nTechnologies/Frameworks: NextJs, React, Express, NodeJs, FastApi, Recoil, Tailwind CSS, Prisma(ORM)\nDatabases: MongoDB, PostgreSQL, MySQL, Redis\nOther Skills: AWS, GCP, Docker, Git, Linux, Bash Shell, Blockchain, Postman, Serverless\nExperience\nTecell Oct 2024 – Present\nSoftware Engineer - Intern Bengaluru, Karnataka\n• Focusing on Node and cloud application integration to develop software that connects and manages EV charging\nstations.\n• Contributing to projects aimed at enhancing system performance and scalability within the EV infrastructure.\n

In [42]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever, "question":  itemgetter("question")
    } 
    | prompt 
    | model
)

chain.invoke({"question": "Who is shrikrushna?"})

"I don't know the answer to that question as there is no clear identification of who Shrikrushna is in the provided context."

In [43]:
questions = [
    "Whats the name of guy in mycv?",
    "What are the projects Shrikrushna made?",
    "give a brief about Shrikrushna'a educational background?",
    "What are his technical skills?"
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")


Question: Whats the name of guy in mycv?
Answer: I don't know the answer to that question as the context provided does not mention the name of the person in mycv. The context only provides information about Shrikrushna Gundre's education, technical skills, experience, and projects he has worked on.
Question: What are the projects Shrikrushna made?
Answer: Based on the provided resume content, Shrikrushna has worked on the following projects:

1. Next Portfolio: Developed a responsive portfolio website using Next.js and TypeScript, showcasing projects and skills with a focus on modern web standards. Implemented animations using Aceternity and Framer Motion for enhanced user interaction.
2. PayTm - Community Payments: Developed a community payments platform using JavaScript and React, enhancing user engagement and transaction efficiency. Implemented backend services with Express and MongoDB, adhering to atomicity standards for secure and reliable transaction processing. Created a respons